In [1]:
#config GPU to not cause any error while training
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
# import required modules
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [5]:
# set required directories
train_dir = "./dataset/train/"
valid_dir = "./dataset/valid/"
IMAGE_DIMS = (256,256)

In [4]:
# create image data generator
datagen = ImageDataGenerator(rescale=1./255)

In [5]:
# load the images from the directories
train_data = datagen.flow_from_directory(train_dir, target_size=IMAGE_DIMS, batch_size=20, class_mode="categorical", color_mode="rgb", shuffle=True)
valid_data = datagen.flow_from_directory(valid_dir, target_size=IMAGE_DIMS, batch_size=20, class_mode="categorical", color_mode="rgb", shuffle=True)

Found 70295 images belonging to 38 classes.
Found 17572 images belonging to 38 classes.


In [6]:
# set required parameters
INPUT_SHAPE = train_data[0][0].shape[1:] # (256,256,3)
NUM_OUTPUT_LAYER = len(train_data.class_indices)

In [8]:
# load pre-trained model
pre_trained_model = InceptionV3(input_shape=INPUT_SHAPE, include_top=False)
for layer in pre_trained_model.layers:
    layer.trainable = False
last_layer = pre_trained_model.get_layer('mixed7').output

In [15]:
# create untrained model using functional API
x = keras.layers.Conv2D(16, (3,3), activation="relu", name="CONV-TRAINABLE-1")(last_layer)
x = keras.layers.MaxPool2D(2,2)(x)
x = keras.layers.Conv2D(32, (3,3), activation="relu", name="CONV-TRAINABLE-2")(x)
x = keras.layers.BatchNormalization(name="NORM-TRAINAABLE")(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(256, activation="relu", name="DENSE_1")(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Dense(128, activation="relu", name="DENSE_2")(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Dense(64, activation="relu", name="DENSE_3")(x)
x = keras.layers.Dense(NUM_OUTPUT_LAYER, activation="softmax", name="OUTPUT_LAYER")(x)

model = keras.Model(pre_trained_model.input, x)
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

In [16]:
# print model summary
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 127, 127, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 127, 127, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 127, 127, 32) 0           batch_normalization[0][0]        
____________________________________________________________________________________________

In [11]:
# create tensorboard callback
tb_callback = keras.callbacks.TensorBoard("logs/v2/", histogram_freq=1, write_images=True, update_freq="batch")

In [17]:
# train the model
model.fit(train_data, validation_data=valid_data, epochs=10, callbacks=[tb_callback])

Epoch 1/10
3515/3515 [==============================] - 409s 114ms/step - loss: 1.1461 - accuracy: 0.6569 - val_loss: 0.3404 - val_accuracy: 0.8860
Epoch 2/10
3515/3515 [==============================] - 346s 98ms/step - loss: 0.2929 - accuracy: 0.9076 - val_loss: 0.1931 - val_accuracy: 0.9410
Epoch 3/10
3515/3515 [==============================] - 312s 89ms/step - loss: 0.2148 - accuracy: 0.9326 - val_loss: 0.2325 - val_accuracy: 0.9311
Epoch 4/10
3515/3515 [==============================] - 334s 95ms/step - loss: 0.1758 - accuracy: 0.9459 - val_loss: 0.2009 - val_accuracy: 0.9418
Epoch 5/10
3515/3515 [==============================] - 345s 98ms/step - loss: 0.1489 - accuracy: 0.9546 - val_loss: 0.1918 - val_accuracy: 0.9410
Epoch 6/10
3515/3515 [==============================] - 330s 94ms/step - loss: 0.1307 - accuracy: 0.9598 - val_loss: 0.2027 - val_accuracy: 0.9434
Epoch 7/10
3515/3515 [==============================] - 327s 93ms/step - loss: 0.1144 - accuracy: 0.9654 - val_loss: 

In [3]:
# save model
model.save("./models/model_v3_b20_trainable_conv.model")

NameError: name 'model' is not defined

In [1]:
# import required modules for testing purpose
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np
import os

In [2]:
# load the model
model = keras.models.load_model("./models/model_v3_b20_trainable_conv.model")

In [3]:
# create function for testing
def predict_image(IMAGE_PATH, classes): # classes = ["D1", "D2", "D3" ...]
    # open and load the image from the path
    img = image.load_img(IMAGE_PATH, target_size=(IMAGE_DIMS), color_mode='rgb')
    
    # convert the image to a numpy array
    x = image.img_to_array(img) # tf.keras.preprocessing.image.Image -> numpy.Array
    
    # normalize the image
    x = np.expand_dims(x, axis=0)/255 # 0 -> 0, 255 -> 1 || divide values by 255

    images = np.vstack([x]) # make a vertical stack of the image

    # predict the class
    test = model(images)

    # get the index of the class with the highest probability
    p_class = np.argmax(test.numpy()[0]) # [2, 3, 6, 3] -> np.argmax() -> 2

    # get the class name
    probabilty = test[0][p_class].numpy()

    # print the class name and the probability
    return classes[p_class], round(probabilty*100, 2)

In [6]:
# testing on an image
test = predict_image("./dataset/test/corn_rust.jpg", os.listdir("./dataset/train"))

In [7]:
test

('Tomato___Target_Spot', 98.17)

In [51]:
# iske aage yaad nahi kya kiya tha
import json
import random

In [52]:
classes = {}
for item in os.listdir("./dataset/train"):
    random.seed(item)
    classes[item] = random.randint(100000, 999999)

In [56]:
json.dump(classes, open("classes.json", "w"), indent=4)